In [1]:
from dask.distributed import Client
client = Client('192.168.9.30:8786')

In [ ]:
from dask_tensorflow import start_tensorflow
tf_spec, dask_spec = start_tensorflow(client, ps=2, worker=4)




In [ ]:
X, y = make_classification(n_samples=600000, n_features=10, random_state=42)

In [ ]:
def build_deep_neural_network():
    # create model
    model = Sequential()
    model.add(Dense(60, input_shape=(10,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:

model = build_deep_neural_network()

model.fit(X, y, verbose=1, epochs=2000)